In [1]:
################################## 1. Load packages, define dc, match gdf and load linescan images. 

from mpl_toolkits.mplot3d import Axes3D #to plot in 3D
from matplotlib import colors           #extra colour representations
from matplotlib import cm               #extra colour representations
import cv2                              #to perform more complex manipulations on arrays

%matplotlib inline
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime

from datacube import Datacube
import matplotlib.pyplot as plt

sys.path.append("../scripts")
from dea_plotting import map_shapefile
from dea_spatialtools import xr_rasterize
import re

/env/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# Define dc!
dc = Datacube(app="Masking")

/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


In [3]:
# idname!
#idname = pd.read_csv('cleanclean.csv', quotechar = '"')
#idname

In [3]:
### GDF

vector_file = 'resources/fire_boundaries.shp'
gdf = gpd.read_file(vector_file)
def clean_name(name):
    if name is None:
        res = None
    else:
        if name.upper()[-4::] == ".JPG":
            res = name.upper()[:-4].replace(' ','_')
        else:
            res = name.upper().replace(' ','_')
    return res

gdf['SourceNameClean'] = gdf.apply(lambda row: clean_name(row.SourceName), axis=1)
gdf.dtUTC = gdf.apply(lambda row: datetime.strptime(row.dtUTC, '%Y-%m-%d %H:%M:%S'), axis=1)
gdf.dtLocal = gdf.apply(lambda row: datetime.strptime(row.dtLocal, '%Y-%m-%d %H:%M:%S'), axis=1)
gdf['Composite'] = gdf.SourceName.str.upper().str.contains(",|&|(COMPOSITE)", na=False)


/env/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [4]:
## Linescan datasets!
linescan_datasets = dc.find_datasets(product='linescan')
linescan_datasets = sorted(linescan_datasets, key = lambda ds: (ds.center_time, ds.id))
linescan_datasets = pd.Series(linescan_datasets)

# linescan 74 and 92

In [8]:
from sklearn.metrics import f1_score

In [14]:

mopen = np.array([1,2,3])
mclose = np.array([64, 150, 300, 500])

#linescan1=np.array([74, 97, 27])
#linescan2=np.array([92, 124, 29])
linescan1=np.array([23])
linescan2=np.array([26])
for ls1, ls2 in zip(linescan1, linescan2):
    for maskopen in mopen :
        for maskclose in mclose:
            print('linescan1: ', ls1, ', linescan2: ', ls2, ', maskopen: ', maskopen, ', maskclose: ', maskclose)

linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  64
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  150
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  300
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  500
linescan1:  23 , linescan2:  26 , maskopen:  2 , maskclose:  64
linescan1:  23 , linescan2:  26 , maskopen:  2 , maskclose:  150
linescan1:  23 , linescan2:  26 , maskopen:  2 , maskclose:  300
linescan1:  23 , linescan2:  26 , maskopen:  2 , maskclose:  500
linescan1:  23 , linescan2:  26 , maskopen:  3 , maskclose:  64
linescan1:  23 , linescan2:  26 , maskopen:  3 , maskclose:  150
linescan1:  23 , linescan2:  26 , maskopen:  3 , maskclose:  300
linescan1:  23 , linescan2:  26 , maskopen:  3 , maskclose:  500


In [15]:
mopen = np.array([1,2,3])
mclose = np.array([64, 150, 300, 500])



for ls1, ls2 in zip(linescan1, linescan2):
    for maskopen in mopen :
        for maskclose in mclose:
            print('linescan1: ', ls1, ', linescan2: ', ls2, ', maskopen: ', maskopen, ', maskclose: ', maskclose)

            ####### Create mask for the first test image. 
            # 1. Plot the first train image
            # 2. Find the closest image from train set()
            # 3. Rasterize the train image
            # 4. Extract points where is brighter than 150 from test data
            # 5. Overlap the train image on train data.
            # 6. Mask the overlapped image to get the prediction
            # 7. Extract the required points for test set


            # 1. Plot the first train image
            ob1  = gdf[gdf.SourceNameClean == 'YARRA51_633_P1_201903061644_MGA94_55']
            src1 = dc.load(product='linescan', id=linescan_datasets[ls1].id, output_crs='epsg:28355', resolution=(-10,10))
            #fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,15))
            #src1.linescan.plot(ax=ax)
            #ob1.geometry.exterior.plot(ax=ax, edgecolor='red', linewidths=1)


            ## 2. Find the closest time and place from training data.
            ob2  = gdf[gdf.SourceNameClean == 'YARRA51_704_P1_201903091659_MGA94_55']
            src2 = dc.load(product='linescan', id=linescan_datasets[ls2].id, output_crs='epsg:28355', resolution=(-10,10))
            #fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,15))
            #src2.linescan.plot(ax=ax)
            #ob1.geometry.exterior.plot(ax=ax, edgecolor='red', linewidths=1)

            ## mask: the true model
            tgt = xr_rasterize(gdf=ob2, da=src2)
            truefire=tgt
            src2['truefire']=truefire


            # 3. Rasterize the train image

            tgt = xr_rasterize(gdf=ob1, da=src1)
            rgb_mask1=tgt*255
            src1['rgb_mask1']=rgb_mask1



            # 4. Extract points where is brighter than 100 from test data

            lowerBound=100
            upperBound=255
            #create the mask over the top of our original image
            rgb_mask2 = cv2.inRange(src2.linescan.values[0],lowerBound,upperBound)

            colname = src2.x.values
            rowname = src2.y.values
            rgb_mask2 = pd.DataFrame(rgb_mask2, index = rowname, columns = colname)
            rgb_mask2.shape


            # 5. Overlap the train image on train data.

            rgb_mask2.index.name = 'y'
            rgb_mask2.columns.name = 'x'
            src2['rgb_mask2'] = rgb_mask2

            dftest=pd.concat([src1.to_dataframe().reset_index(), src2.to_dataframe().reset_index()], ignore_index=True, sort=False)
            dftest['rgb_mask3']=np.fmax(dftest.loc[:,'rgb_mask1'], dftest.loc[:,'rgb_mask2'])
            newdf=dftest.drop(columns=['time','linescan','spatial_ref', 'rgb_mask1', 'rgb_mask2'], axis=1)
            dates = '2019-01-27~28'
            x = newdf.x
            y = newdf.y


            overlap_rgb = newdf.rgb_mask3
            df_multiindex = pd.DataFrame({'date': dates,
                                          'x': x,
                                          'y': y,
                                          'overlap_rgb': overlap_rgb})
            df_multiindex = df_multiindex.set_index(['date', 'x','y'])
            rgb_mask=df_multiindex[~df_multiindex.index.duplicated()].to_xarray()

            fillna=rgb_mask.fillna(value=0)

            #fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,15))
            #fillna.overlap_rgb.plot(ax=ax)



            # 6. Mask the overlapped image to get the prediction

            #establish dialation and contraction parameters
            kernelOpen=np.ones((maskopen,maskopen)) # try it yourself!
            kernelClose=np.ones((maskclose,maskclose))      

            #denoise the pixels
            maskOpen=cv2.morphologyEx(fillna.overlap_rgb[0].values,cv2.MORPH_OPEN,kernelOpen)

            #plot the denoised pixels
            #join the pxels together
            maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

            #fig = plt.figure(figsize=(20, 10))
            #plt.imshow(maskClose)
            #plt.show()
            #establish dialation and contraction parameters

            # 3. Compare prediction and true fire area

            ## mask: the prediction
            our_mask = fillna.overlap_rgb
            our_mask.values[0] = maskClose/255

            #prediction=our_mask.sel(x=slice(src2.x.min(), src2.x.max()), y=slice(src2.y.min(), src2.y.max()))


            # Make testset that we will compare the true-fire and our pred.round()iction

            x=np.linspace(src2.x.min(), src2.x.max(), num=100)
            y=np.linspace(src2.y.min(), src2.y.max(), num=100)
            x_dp=np.repeat(x,100)
            y_dp=np.tile(y,100)
            testset = pd.DataFrame({'x':x_dp, 'y':y_dp})


            # iterate over the coordinates that are required for testing in the current linescan file
            for idx, ob in testset.iterrows():
                result_tf = src2.sel(x=ob.x, y=ob.y, method='nearest').truefire.values
                result_pd = our_mask.sel(x=ob.x, y=ob.y, method='nearest')
                result_10 = int(result_tf == 1)
                result_11 = int(result_pd == 1)
                testset.loc[(testset.x==ob.x) & (testset.y==ob.y), 'truefire'] = result_10
                testset.loc[(testset.x==ob.x) & (testset.y==ob.y), 'prediction'] = result_11


            # Calculate F1 score
            pred=testset.truefire
            true=testset.prediction



            f1score=f1_score(true, pred, average='macro')
            print('linescan1: ', ls1, ', linescan2: ', ls2, ', maskopen: ', maskopen, ', maskclose: ', maskclose, ', f1 score: ', f1score)


linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  64
Rasterizing to match xarray.DataArray dimensions (1297, 3333)
Rasterizing to match xarray.DataArray dimensions (777, 1171)
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  64 , f1 score:  0.4999749987499375
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  150
Rasterizing to match xarray.DataArray dimensions (1297, 3333)
Rasterizing to match xarray.DataArray dimensions (777, 1171)
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  150 , f1 score:  0.4999749987499375
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  300
Rasterizing to match xarray.DataArray dimensions (1297, 3333)
Rasterizing to match xarray.DataArray dimensions (777, 1171)
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  300 , f1 score:  0.4999749987499375
linescan1:  23 , linescan2:  26 , maskopen:  1 , maskclose:  500
Rasterizing to match xarray.DataArray dimensions (1297, 3333)
Rasterizing to match

# 23 and 26